In [1]:
from langchain_community.document_loaders import YoutubeLoader

try:
    loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False)
    docs = loader.load()
    print(type(docs))
    
except Exception as e:
    print(e)

no element found: line 1, column 0


In [2]:
from langchain_community.document_loaders import YoutubeLoader

video_id = "5MgBikgcWnY"  # TEDx video
docs = []
try:
    
    loader = YoutubeLoader.from_youtube_url(
        "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
    )
    docs = loader.load()
    print(docs)

except Exception as e:
    print("❌ Failed to load transcript.")
    print(type(e).__name__ + ":", e)


❌ Failed to load transcript.
ParseError: no element found: line 1, column 0


In [30]:
len("\n".join([line["text"] for line in transcript]))

14918

In [108]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
)

loader.load()

ParseError: no element found: line 1, column 0 (<string>)

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "5MgBikgcWnY"  # Example: TEDx - How to Learn Anything

try:
    # List all available transcripts
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    print(type(transcript_list))
    
except Exception as e:
    print("❌ Could not fetch transcript.")
    print(type(e).__name__ + ":", e)


❌ Could not fetch transcript.
ParseError: no element found: line 1, column 0


In [112]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript_chunks(video_id, chunk_size=500):
    try:
        # Fetch transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        
        # Break into chunks
        chunks = []
        current_chunk = ""
        for i, line in enumerate(transcript):
            text = line['text']
            # Add sentence to current chunk
            if len(current_chunk) + len(text) + 1 <= chunk_size:
                current_chunk += " " + text
            else:
                chunks.append(current_chunk.strip())
                current_chunk = text

        # Add last chunk
        if current_chunk:
            chunks.append(current_chunk.strip())

        print(f"✅ Transcript split into {len(chunks)} chunks.")
        return chunks

    except Exception as e:
        print("❌ Failed to fetch transcript.")
        print(type(e).__name__ + ":", e)
        return []

# Example usage
video_id = "5MgBikgcWnY"  # Replace with your video ID
chunks = get_transcript_chunks(video_id, chunk_size=1000)

# Print first 2 chunks for preview
for i, chunk in enumerate(chunks[:2]):
    print(f"\n--- Chunk {i+1} ---\n{chunk}\n")


❌ Failed to fetch transcript.
ParseError: no element found: line 1, column 0
